In [2]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import os

# ====== 建立 Scraper ======
scraper = cloudscraper.create_scraper()
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Referer": "https://finfo.tw"
}

# ====== 設定 ======
keywords = ["產險", "壽險", "投資型"]
max_pages = 100
base_url = "https://finfo.tw"
output_file = "finfo_posts_產險_壽險_投資型.csv"

# ====== 擷取單篇貼文與留言 ======
def extract_post_data(post_url, keyword):
    print(f"🔍 擷取貼文：{post_url}")
    res = scraper.get(post_url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    post_block = soup.select_one("div.post-content")
    author_block = soup.select_one("span.user-name") or soup.select_one("div.d-flex.align-items-center span")

    post_text = post_block.get_text(strip=True) if post_block else ""
    author = author_block.get_text(strip=True) if author_block else ""
    post_time = ""
    time_match = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}', res.text)
    if time_match:
        post_time = time_match.group()

    results = [{
        "keyword": keyword,
        "url": post_url,
        "type": "貼文",
        "author": author,
        "content": post_text,
        "time": post_time
    }]

    # 留言
    comments = soup.select("div.d-flex.justify-content-between.align-items-center")
    for comment in comments:
        user = comment.select_one("span.text-gray-600")
        text = comment.select_one("div.text-gray-800")
        time_tag = comment.select_one("span.text-gray-500")

        results.append({
            "keyword": keyword,
            "url": post_url,
            "type": "留言",
            "author": user.get_text(strip=True) if user else "",
            "content": text.get_text(strip=True) if text else "",
            "time": time_tag.get_text(strip=True) if time_tag else ""
        })

    time.sleep(1)
    return results

# ====== 主流程 ======
if os.path.exists(output_file):
    print(f"📄 發現舊檔案 {output_file}，將以追加方式寫入")
else:
    print(f"📄 建立新檔案：{output_file}")

for keyword in keywords:
    print(f"\n🚀 開始爬關鍵字：{keyword}")
    for page in range(1, max_pages + 1):
        print(f"📄 處理列表頁第 {page} 頁")
        list_url = f"https://finfo.tw/posts?button=&page={page}&post_q%5Bkeyword%5D={keyword}"

        try:
            res = scraper.get(list_url, headers=headers)
        except Exception as e:
            print(f"❌ 請求失敗，跳過此頁：{e}")
            continue

        if res.status_code != 200 or "沒有符合條件的貼文" in res.text:
            print("✅ 無更多結果，跳出此關鍵字")
            break

        soup = BeautifulSoup(res.text, "html.parser")
        links = soup.select('a.text-decoration-none.d-flex.justify-content-center.row[href^="/posts/"]')
        if not links:
            print("⚠️ 找不到貼文連結，跳出此關鍵字")
            break

        for link in links:
            href = link.get("href")
            full_url = base_url + href
            try:
                post_results = extract_post_data(full_url, keyword)

                # 立即 append 存入 CSV
                df_temp = pd.DataFrame(post_results)
                df_temp.to_csv(output_file, mode='a', index=False, encoding='utf-8-sig',
                               header=not os.path.exists(output_file) or os.stat(output_file).st_size == 0)
            except Exception as e:
                print(f"❌ 擷取失敗 {full_url}：{e}")

print(f"\n✅ 所有關鍵字完成，結果已存入 {output_file}")


📄 建立新檔案：finfo_posts_產險_壽險_投資型.csv

🚀 開始爬關鍵字：產險
📄 處理列表頁第 1 頁
🔍 擷取貼文：https://finfo.tw/posts/10154
🔍 擷取貼文：https://finfo.tw/posts/9779
🔍 擷取貼文：https://finfo.tw/posts/11906
🔍 擷取貼文：https://finfo.tw/posts/10209
🔍 擷取貼文：https://finfo.tw/posts/9990
🔍 擷取貼文：https://finfo.tw/posts/13908
🔍 擷取貼文：https://finfo.tw/posts/13581
🔍 擷取貼文：https://finfo.tw/posts/11544
🔍 擷取貼文：https://finfo.tw/posts/13590
🔍 擷取貼文：https://finfo.tw/posts/12328
🔍 擷取貼文：https://finfo.tw/posts/11117
🔍 擷取貼文：https://finfo.tw/posts/14049
🔍 擷取貼文：https://finfo.tw/posts/13275
🔍 擷取貼文：https://finfo.tw/posts/13561
🔍 擷取貼文：https://finfo.tw/posts/12192
🔍 擷取貼文：https://finfo.tw/posts/11243
🔍 擷取貼文：https://finfo.tw/posts/13833
🔍 擷取貼文：https://finfo.tw/posts/13027
🔍 擷取貼文：https://finfo.tw/posts/10969
🔍 擷取貼文：https://finfo.tw/posts/9985
📄 處理列表頁第 2 頁
🔍 擷取貼文：https://finfo.tw/posts/8782
🔍 擷取貼文：https://finfo.tw/posts/7953
🔍 擷取貼文：https://finfo.tw/posts/13691
🔍 擷取貼文：https://finfo.tw/posts/9332
🔍 擷取貼文：https://finfo.tw/posts/6653
🔍 擷取貼文：https://finfo.tw/posts/1381

In [3]:
# 重新載入使用者剛上傳的 finfo 檔案並查看資料筆數與欄位
finfo_df = pd.read_csv("finfo_posts_產險_壽險_投資型.csv")
row_count = finfo_df.shape[0]
columns = finfo_df.columns.tolist()
row_count, columns


(225179, ['keyword', 'url', 'type', 'author', 'content', 'time'])